In [ ]:
%load_ext autoreload
%autoreload 2

# AI Lab tools/utils

In [ ]:
import ailab as lab

In [8]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Read Data

In [1]:
import pandas as pd

feature_names=["Age", "Workclass", "Final Weight", "Education", "Education-Num", "Marital Status",
               "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
               "Hours per week", "Country", "Income"]

df_train=pd.read_csv("../../data/adult.data",
                     names=feature_names)

df_test=pd.read_csv("../../data/adult.test",
                    skiprows =1,
                    names=feature_names)




# Test set prep

In [2]:
def df_prep(df):    
    LABEL="Income"
    y_train=df[LABEL].replace([" <=50K"," <=50K."," >50K"," >50K."],[0,0,1,1])
    df_X=df.drop(LABEL, axis=1)
    return df_X,y_train

df_X_train,y_train=df_prep(df_train)
df_X_test,y_test=df_prep(df_test)

In [3]:
eval_df=df_test

# Score models in folder

**Warning: Loading pickled objects from untrusted origin is not safe -*allows remote execution!*-**

In [14]:
import glob
import os
import joblib

models=glob.glob("../../models/*.pickle",recursive=True)
score_cols=[]
list_predictions=[]
for path_model in models:    
    model_id=os.path.basename(path_model).split(".")[0]
    logger.info(f"Scoring {model_id}...")
    model_pipeline=joblib.load(path_model)
    model_score_col=model_id
    score_cols.append(model_score_col)
    eval_df[model_score_col]=model_pipeline.predict_proba(df_X_test)[:,1]

eval_df

INFO:__main__:Scoring adult_logreg_default...
INFO:__main__:Scoring adult_logreg_pca...
INFO:__main__:Scoring adult_randomforest_default...
INFO:__main__:Scoring adult_randomforest_full...


,Age,Workclass,Final Weight,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,...,Capital Loss,Hours per week,Country,Income,adult_logreg_default,adult_logreg_pca,adult_randomforest_default,adult_randomforest_full,y,index
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,...,0,40,United-States,<=50K.,0.199906,0.006242,0.0,0.005964,0,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,...,0,50,United-States,<=50K.,0.260422,0.225803,0.2,0.521223,0,1
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,...,0,40,United-States,>50K.,0.141116,0.391106,0.5,0.585346,1,2
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,...,0,40,United-States,>50K.,0.795099,0.826285,0.8,0.902968,1,3
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,...,0,30,United-States,<=50K.,0.309022,0.007372,0.0,0.002405,0,4
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,...,0,30,United-States,<=50K.,0.226127,0.005802,0.0,0.036616,0,5
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,...,0,40,United-States,<=50K.,0.195888,0.009344,0.0,0.023196,0,6
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,...,0,32,United-States,>50K.,0.496662,0.868201,0.7,0.843202,1,7
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,...,0,40,United-States,<=50K.,0.128317,0.018273,0.0,0.023817,0,8
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,...,0,10,United-States,<=50K.,0.314226,0.047919,0.0,0.228314,0,9


# Prepare dataframe for Dash

In [6]:
eval_df=df_test

# these two will be hidden from UI, below
eval_df["y"]=y_test
eval_df["index"]=eval_df.index.values
eval_df

,Age,Workclass,Final Weight,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,...,Capital Loss,Hours per week,Country,Income,adult_logreg_default,adult_logreg_pca,adult_randomforest_default,adult_randomforest_full,y,index
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,...,0,40,United-States,<=50K.,0.199906,0.006242,0.0,0.005964,0,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,...,0,50,United-States,<=50K.,0.260422,0.225803,0.2,0.521223,0,1
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,...,0,40,United-States,>50K.,0.141116,0.391106,0.5,0.585346,1,2
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,...,0,40,United-States,>50K.,0.795099,0.826285,0.8,0.902968,1,3
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,...,0,30,United-States,<=50K.,0.309022,0.007372,0.0,0.002405,0,4
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,...,0,30,United-States,<=50K.,0.226127,0.005802,0.0,0.036616,0,5
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,...,0,40,United-States,<=50K.,0.195888,0.009344,0.0,0.023196,0,6
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,...,0,32,United-States,>50K.,0.496662,0.868201,0.7,0.843202,1,7
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,...,0,40,United-States,<=50K.,0.128317,0.018273,0.0,0.023817,0,8
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,...,0,10,United-States,<=50K.,0.314226,0.047919,0.0,0.228314,0,9


# Dash 

In [11]:
import sklearn.metrics as metrics
import plotly.graph_objs as go

def serve_roc_curves(df,y_test_col,group_by=None,split_by=None,agg_metric="mean"):
    
    figures=[]
    
    unpivot_df=pd.melt(df, id_vars=set(eval_df.columns.values)-set(score_cols), 
                       var_name='Model', value_name='Prob')
    
    # Add group by all column (used if no groupby/split specified)
    unpivot_df["all"]=""
    
    if not group_by:
        group_by="all"
    if not split_by:
        split_by="all"
        
    for i_split,i_split_data in unpivot_df.groupby(split_by):
        # A chart each
        traces=[]
        for i_group,i_group_data in i_split_data.groupby(group_by):
            # a trace in a chart each
            try:
                y_score=pd.DataFrame(i_group_data.groupby("index")["Prob"].agg(agg_metric))
                y_test=df.loc[y_score.index].y
                
                fpr, tpr, threshold = metrics.roc_curve(y_test, y_score)
                auc_score = metrics.roc_auc_score(y_true=y_test, y_score=y_score)

                trace = go.Scatter(
                    x=fpr,
                    y=tpr,
                    mode='lines',
                    name=f"{i_group}:{auc_score:.3f}",
                )
                traces.append(trace)
            except Exception as error:
                logger.error(error)

        title=f'-{i_split}-'
        if group_by=="all":
            title=f"ROC AUC:{auc_score:.3f}"
            
        layout = go.Layout(
                title=title,
                xaxis=dict(
                    title='False Positive Rate'
                ),
                yaxis=dict(
                    title='True Positive Rate'
                ),
                margin=dict(l=50, r=10, t=55, b=40),
            )

        figure = go.Figure(data=traces, layout=layout)

        figures.append((title,figure))

    return figures

# Test
serve_roc_curves(eval_df,"y","Model","Workclass")


C:\tools\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning:

No positive samples in y_true, true positive value should be meaningless

ERROR:__main__:Only one class present in y_true. ROC AUC score is not defined in that case.
ERROR:__main__:Only one class present in y_true. ROC AUC score is not defined in that case.
ERROR:__main__:Only one class present in y_true. ROC AUC score is not defined in that case.
ERROR:__main__:Only one class present in y_true. ROC AUC score is not defined in that case.


[('- ?-',
  {'data': [{'type': 'scatter',
     'x': array([0.        , 0.        , 0.00224972, 0.00224972, 0.00674916,
            0.00674916, 0.00899888, 0.00899888, 0.01237345, 0.01237345,
            0.02249719, 0.02249719, 0.02924634, 0.02924634, 0.03262092,
            0.03262092, 0.03374578, 0.03374578, 0.04724409, 0.04724409,
            0.04949381, 0.04949381, 0.05174353, 0.05174353, 0.07424072,
            0.07424072, 0.08211474, 0.08211474, 0.14848144, 0.14848144,
            0.15973003, 0.15973003, 0.20247469, 0.20247469, 0.22384702,
            0.22384702, 0.23172103, 0.23172103, 0.27334083, 0.27334083,
            0.32733408, 0.32733408, 0.34870641, 0.34870641, 0.36107987,
            0.36107987, 0.37007874, 0.37007874, 0.41169854, 0.41169854,
            0.42519685, 0.42519685, 0.44656918, 0.44656918, 0.46119235,
            0.46119235, 0.4904387 , 0.4904387 , 0.50618673, 0.50618673,
            0.51293588, 0.51293588, 0.53880765, 0.53880765, 0.53993251,
            0.539

In [12]:
import dash
from dash.dependencies import Input, Output, State,Event
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dt
import json
import plotly
from IPython import display

app = dash.Dash()

# Reference: https://github.com/plotly/dash-svm/
external_css = [
    # Normalize the CSS
    "https://cdnjs.cloudflare.com/ajax/libs/normalize/7.0.0/normalize.min.css",
    # Fonts
    "https://fonts.googleapis.com/css?family=Open+Sans|Roboto",
    "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
    # Base Stylesheet, replace this with your own base-styles.css using Rawgit
    "https://rawgit.com/xhlulu/9a6e89f418ee40d02b637a429a876aa9/raw/f3ea10d53e33ece67eb681025cedc83870c9938d/base-styles.css",
    # Custom Stylesheet, replace this with your own custom-styles.css using Rawgit
    "https://cdn.rawgit.com/plotly/dash-svm/bb031580/custom-styles.css"
]

for css in external_css:
    app.css.append_css({"external_url": css})

    
visible_cols=eval_df.columns.drop(["index","y"]).values
dropdown_cols=["Model"]+list(eval_df.select_dtypes("object").columns.values)


app.layout = html.Div([
    # Reference: https://github.com/plotly/dash-svm/
    # .container class is fixed, .container.scalable is scalable
    html.Div(className="banner", children=[
        html.Div(className='container scalable', children=[
            html.H2(html.A(
                'Dash Classification Eval - DevScope AI Lab',
                href='https://github.com/DevScope/ai-lab',
                style={
                    'text-decoration': 'none',
                    'color': 'inherit'
                }
            )),

            html.A(
                html.Img(src="https://s3-us-west-1.amazonaws.com/plotly-tutorials/logo/new-branding/dash-logo-by-plotly-stripe-inverted.png"),
                href='https://plot.ly/products/dash/'
            )
        ]),
    ]),
    html.Div(id='body', className='container scalable', children=[
        
         html.Div([
            html.Div(
                [
                    dt.DataTable(
                        rows=eval_df.to_dict('records'),
                        editable=False,
                        sortable=True,
                        columns=visible_cols,
                        row_selectable=False,
                        filterable=True,
                        id='score_table'
                        ),
                ],className="six columns"),
             html.Div(
                 [ html.Div([
                     dcc.Dropdown(
                         id='group_by',
                         options=[{'label':label,'value':label} for label in dropdown_cols],
                         value="Model",
                         placeholder="Group ROC by column",
                         className="six columns"
                        ),
                     dcc.Dropdown(
                         id='split_by',
                         options=[{'label':label,'value':label} for label in dropdown_cols],
                         value=None,
                         placeholder="Split Charts by column",
                         className="six columns"
                        ),
                         ],className="row"),
                        html.Div(id="output")
                   ],id="results",className="six columns",style={'height':'500px','overflow-y': 'scroll'})
        ],className="row")
     ])
])

@app.callback(
   Output('output', 'children'),
   [Input("score_table","rows"),Input("group_by","value"),Input("split_by","value")])
def update_feature_table(rows,group_by,split_by):
    if len(rows)==0:
        return []
    
    children=[]
    
    #rebuild dataframe from received data
    df_selected = pd.DataFrame(rows)
    df_selected.index=df_selected["index"]
    
    roc_figures = serve_roc_curves(df_selected,"y",group_by,split_by)
    
    for title,i_roc_figures in roc_figures:
        children.append(dcc.Graph(
                    id=title,
                    figure=i_roc_figures,
                    config={ 'displayModeBar': False}))
    return children
    
    
    

In [15]:
# use <esc> i+i on Jupyter to quick interrupt & get control back to jupyter
lab.show_app(app)


Open in new window

INFO:werkzeug: * Running on http://0.0.0.0:10001/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [29/Aug/2018 00:22:44] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2018 00:23:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2018 00:23:11] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2018 00:23:11] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2018 00:23:11] "GET /favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2018 00:23:14] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2018 00:24:41] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2018 00:24:45] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2018 00:25:25] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2018 00:25:33] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:wer